# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [12]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI  
from IPython.display import Markdown, display, update_display
import ollama
from httpx import stream

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [11]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()
system_prompt = """you are the best in the world in explaining code to a beginner who is new to coding. 
consider the question provided by the user and explain it in a way that is easy to understand.
use examples and analogies to help explain the question. Display in markdown without code blocks"""


API key looks good so far


In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [13]:
# Get gpt-4o-mini to answer, with streaming


stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        stream=True
    )
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

Sure! Let's break down the code you provided step by step.

### Understanding the Structure

1. **`yield from`**:
   - This is a special keyword in Python used within a generator function. A generator function is a function that uses `yield` to return values one at a time, instead of returning them all at once.
   - When you use `yield from`, you are telling Python to yield all values from the iterable that follows it. Think of it as saying, "Give me everything from this container".

2. **`{ ... for book in books if book.get("author")}`**:
   - This is a **set comprehension**, which is a concise way to create a set (a collection of unique items).
   - Inside the curly braces `{}`, we define how we want to create the set:
     - `for book in books`: This part says, "For each `book` in the collection `books`".
     - `if book.get("author")`: This checks if the current `book` has an "author" key, and that it is not `None` or empty. The `get` function is used to safely extract the value of the "author" key, and if it doesn't exist, it won't raise an error. It simply returns `None`.

### Bringing It All Together

Now, let's put everything together to see how it works.

Imagine you have a bookshelf (which we’ll call `books`), and on that shelf, there are different books (each represented as a dictionary with various pieces of information, like "title" and "author"). For example:

```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": None},
    {"title": "Book C", "author": "Author 2"},
    {"title": "Book D", "author": "Author 1"},
]
```

Now, the code you provided will do the following:

- It will look through each book on the bookshelf.
- It checks if each book has a valid "author". If it does, it adds that "author" to a set (to avoid duplicates).
- Then, it yields each unique author found in the books.

### Example

For the above `books` list, if we run your code, the set comprehension would create a set like this:

```python
{ "Author 1", "Author 2" }
```

Finally, when `yield from` is used, it will yield each of these authors one by one. 

### Analogy

Think of your code like this:
- You have a box of fruits (the `books`), and you're looking for apples (the "authors").
- You don't want any rotten apples (books without authors), so you only pick the good ones.
- After you've gathered all the good apples, you put them in a basket (the set), ensuring you don't have duplicates. 
- Finally, with `yield from`, you take one apple out at a time to show someone (yielding each author one by one).

### Conclusion

So, in simple terms, this code extracts all unique authors from a list of books, ignoring any books that don't have an author, and makes them available one at a time for further processing.

In [21]:
# Get Llama 3.2 to answer

response = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
    )
reply = response['message']['content']
display(Markdown(reply))

**Unpacking a List of Books**
=====================================

Let's break down this line of code together.

The code is using a concept called "yield" which allows us to generate a sequence of values instead of computing them all at once and returning them in a list.

```text
yield from {book.get("author") for book in books if book.get("author")}
```

Here's what each part does:

*   `{...}`: This is called a dictionary comprehension. It creates a new dictionary with the results of some operation applied to each item in an input iterable.
    *   `for book in books`: We're iterating over a list of books (`books`).
    *   `if book.get("author")`: We're only considering books that have an "author" field. If a book doesn't have this field, we skip it.
*   `{book.get("author")}`: This takes the author name from each book and puts it in a new dictionary with a single key-value pair.

**The `yield from` Part**
-------------------------

Now, let's talk about `yield from`. It's used to yield items from another iterable. Think of it like taking an item from a bag and putting it on a conveyor belt for someone else to take. The original bag (the one with the books) is not emptied when we take something out; instead, we give that thing to whoever's operating the conveyor belt.

So `yield from` allows us to create a sequence of items from another iterable without having to load them all into memory at once.

**Putting It All Together**
---------------------------

In summary, this line of code takes each book in our list, filters out any books that don't have an author, and then generates a sequence where each item is the author name for a particular book. The `yield from` keyword allows us to do this efficiently without having to store all the authors in memory at once.

For example, if we had a list of books like this:

```python
books = [
    {"title": "Harry Potter", "author": "J.K. Rowling"},
    {"title": "Lord of the Rings", "author": "J.R.R. Tolkien"},
    {"title": "The Great Gatsby", "author": None},
]
```

Running the line of code would give us:

```python
[
    'J.K. Rowling',
    'J.R.R. Tolkien'
]
```

This way, we can use a loop to iterate over each author's name in the list without having to store all the book data in memory at once.